In [15]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [16]:
# load data

df=pd.read_csv('Wilt_withoutdupl_norm_05.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan,outlier
0,0.656711,0.050984,0.044570,0.218476,0.132110,1
1,0.680591,0.049425,0.041939,0.177275,0.106749,1
2,0.734892,0.047396,0.042926,0.259034,0.143741,1
3,0.698087,0.035317,0.027066,0.127065,0.095697,1
4,0.738927,0.046076,0.040228,0.295501,0.112481,1


In [17]:
df.shape

(4819, 6)

In [18]:
# data to values
df = df.values

In [19]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Wilt.json")
gc = GCForest(config)

In [20]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,5]
X = df[:,0:5]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
# number of class
len(np.unique(y_train))

2

In [22]:
             # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 15:39:50,869][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 5)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 5)],y_test.shape=(1591,)
[ 2018-04-26 15:39:50,871][cascade_classifier.fit_transform] group_dims=[5]
[ 2018-04-26 15:39:50,873][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:39:50,874][cascade_classifier.fit_transform] group_ends=[5]
[ 2018-04-26 15:39:50,876][cascade_classifier.fit_transform] X_train.shape=(3228, 5),X_test.shape=(1591, 5)
[ 2018-04-26 15:39:50,877][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 5), X_cur_test.shape=(1591, 5)
[ 2018-04-26 15:39:51,692][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.45%
[ 2018-04-26 15:39:52,754][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.90%
[ 2018-04-26 15:39:53,832][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

In [23]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:40:01,974][cascade_classifier.transform] X_groups_test.shape=[(1591, 5)]
[ 2018-04-26 15:40:01,976][cascade_classifier.transform] group_dims=[5]
[ 2018-04-26 15:40:01,977][cascade_classifier.transform] X_test.shape=(1591, 5)
[ 2018-04-26 15:40:01,979][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 5)


Test Accuracy of ExtraTrees = 98.177247 %


In [24]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 15:40:04,414][cascade_classifier.transform] X_groups_test.shape=[(1591, 5)]
[ 2018-04-26 15:40:04,415][cascade_classifier.transform] group_dims=[5]
[ 2018-04-26 15:40:04,417][cascade_classifier.transform] X_test.shape=(1591, 5)
[ 2018-04-26 15:40:04,419][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 5)


In [25]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 15:40:06,911][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 7)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 7)],y_test.shape=(1591,)
[ 2018-04-26 15:40:06,913][cascade_classifier.fit_transform] group_dims=[7]
[ 2018-04-26 15:40:06,915][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:40:06,917][cascade_classifier.fit_transform] group_ends=[7]
[ 2018-04-26 15:40:06,918][cascade_classifier.fit_transform] X_train.shape=(3228, 7),X_test.shape=(1591, 7)
[ 2018-04-26 15:40:06,920][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 7), X_cur_test.shape=(1591, 7)
[ 2018-04-26 15:40:07,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.76%
[ 2018-04-26 15:40:08,621][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-26 15:40:09,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

In [26]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:40:17,054][cascade_classifier.transform] X_groups_test.shape=[(1591, 7)]
[ 2018-04-26 15:40:17,056][cascade_classifier.transform] group_dims=[7]
[ 2018-04-26 15:40:17,057][cascade_classifier.transform] X_test.shape=(1591, 7)
[ 2018-04-26 15:40:17,059][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 7)


Test Accuracy of ExtraTrees = 98.554368 %


In [27]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [28]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 15:40:19,577][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 7)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 7)],y_test.shape=(1591,)
[ 2018-04-26 15:40:19,579][cascade_classifier.fit_transform] group_dims=[7]
[ 2018-04-26 15:40:19,581][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:40:19,582][cascade_classifier.fit_transform] group_ends=[7]
[ 2018-04-26 15:40:19,584][cascade_classifier.fit_transform] X_train.shape=(3228, 7),X_test.shape=(1591, 7)
[ 2018-04-26 15:40:19,585][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 7), X_cur_test.shape=(1591, 7)
[ 2018-04-26 15:40:20,400][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.07%
[ 2018-04-26 15:40:21,608][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-26 15:40:22,556][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

('Test Accuracy of ExtraTrees = 98.742929 %', 0, 'fois')
(' Time ', '10.795', ' seconds')


[ 2018-04-26 15:40:33,661][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.76%
[ 2018-04-26 15:40:34,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-26 15:40:35,679][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.83%
[ 2018-04-26 15:40:36,748][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.76%
[ 2018-04-26 15:40:37,856][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.45%
[ 2018-04-26 15:40:39,063][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.14%
[ 2018-04-26 15:40:40,140][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.76%
[ 2018-04-26 15:40:41,275][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.45%


('Test Accuracy of ExtraTrees = 98.617222 %', 1, 'fois')
(' Time ', '10.913', ' seconds')


[ 2018-04-26 15:40:47,305][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.76%
[ 2018-04-26 15:40:48,326][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.21%
[ 2018-04-26 15:40:49,393][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.83%
[ 2018-04-26 15:40:50,477][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.07%
[ 2018-04-26 15:40:51,492][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.45%
[ 2018-04-26 15:40:52,557][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.14%
[ 2018-04-26 15:40:53,463][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.76%
[ 2018-04-26 15:40:54,447][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.14%


...

('Test Accuracy of ExtraTrees = 98.554368 %', 17, 'fois')
(' Time ', '10.576', ' seconds')


[ 2018-04-26 15:44:18,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-26 15:44:19,139][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-26 15:44:20,208][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.21%
[ 2018-04-26 15:44:21,332][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.14%
[ 2018-04-26 15:44:22,532][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.45%
[ 2018-04-26 15:44:23,645][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.21%
[ 2018-04-26 15:44:24,751][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.14%
[ 2018-04-26 15:44:25,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.45%


('Test Accuracy of ExtraTrees = 98.617222 %', 18, 'fois')
(' Time ', '11.219', ' seconds')


[ 2018-04-26 15:44:31,708][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-26 15:44:32,837][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-26 15:44:33,946][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.59%
[ 2018-04-26 15:44:35,041][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.45%
[ 2018-04-26 15:44:36,144][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.76%
[ 2018-04-26 15:44:37,240][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.21%
[ 2018-04-26 15:44:38,358][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.83%
[ 2018-04-26 15:44:39,467][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.52%


('Test Accuracy of ExtraTrees = 98.680075 %', 19, 'fois')
(' Time ', '11.021', ' seconds')


# ('Test Accuracy of ExtraTrees = 98.931490 %', 14, 'fois')

# (' Time ', '10.795', ' seconds')